In [1]:
# Import bibliotek
import pandas as pd
import math
import plotly.express as px

In [2]:
# Import danych w formacie CSV
data = pd.read_csv("datasets/dataset.txt", sep=";")

# Wyluskanie danych potrzebnych do analizy i przygotowanie zbioru do pracy
data = data[["Stimulus", "Participant", "Category Right", "Index Right", "Point of Regard Right X [px]", "Point of Regard Right Y [px]"]]

data = data[data["Category Right"] == "Fixation"]
data = data[["Stimulus", "Participant", "Index Right", "Point of Regard Right X [px]", "Point of Regard Right Y [px]"]]
data = data[data["Stimulus"].isin(["Grid Page 1", "Carousel Page 1", "List Page 1"])]
data = data.rename(columns={"Index Right": "Fixation Index"}).reset_index()
data = data.drop("index", axis=1)
data["Fixation Index"] = data["Fixation Index"].astype(int)
data["Point of Regard Right X [px]"] = data["Point of Regard Right X [px]"].astype(float)
data["Point of Regard Right Y [px]"] = data["Point of Regard Right Y [px]"].astype(float)
data['Stimulus'] = data['Stimulus'].str.replace(' Page 1', '')
data = data[data["Participant"] != "test"]
data = data[(data["Point of Regard Right X [px]"] > 0 ) & (data["Point of Regard Right Y [px]"] > 0)]
data

,Stimulus,Participant,Fixation Index,Point of Regard Right X [px],Point of Regard Right Y [px]
0,List,SK,1,816.5,422.1
1,List,SK,1,818.3,420.9
2,List,SK,1,821.0,420.7
3,List,SK,1,820.1,419.8
4,List,SK,1,820.6,418.7
...,...,...,...,...,...
73491,Carousel,KU,8,1180.9,545.0
73492,Carousel,KU,8,1182.5,545.0
73493,Carousel,KU,8,1181.9,545.9
73494,Carousel,KU,8,1179.6,545.9


In [3]:
# Pobranie nazw uczestnikow badania
participants_names = data["Participant"].unique().tolist()

# Pobranie nazw stimulusow
stimulus_names = data["Stimulus"].unique().tolist()
stats = []

# Pobranie ilosci fiksacji dla kazdego uzytkownika
for name in participants_names:
    t = [name]
    
    for stimulus in stimulus_names:
        f_num = len(data[(data["Stimulus"] == stimulus) & (data["Participant"] == name)]["Fixation Index"].unique())
        t.append((stimulus, f_num))
    
    stats.append(t)

# Wyswietlenie zbiorczych statystyk uczestnikow     
display(stats)

[['SK', ('List', 19), ('Carousel', 19), ('Grid', 16)],
 ['AM', ('List', 7), ('Carousel', 6), ('Grid', 11)],
 ['MK', ('List', 21), ('Carousel', 22), ('Grid', 19)],
 ['DM', ('List', 20), ('Carousel', 24), ('Grid', 17)],
 ['PL', ('List', 22), ('Carousel', 23), ('Grid', 22)],
 ['PK', ('List', 5), ('Carousel', 6), ('Grid', 8)],
 ['BM', ('List', 2), ('Carousel', 2), ('Grid', 18)],
 ['FK', ('List', 21), ('Carousel', 25), ('Grid', 26)],
 ['AT', ('List', 20), ('Carousel', 24), ('Grid', 27)],
 ['SM', ('List', 17), ('Carousel', 21), ('Grid', 20)],
 ['KU', ('List', 10), ('Carousel', 8), ('Grid', 10)]]

In [4]:
def calculate(x, y, stim):
    
    # Rozmiary strony oraz ekranu
    x_res = 1680
    y_res = 1050
    page_width = 1920
    page_height = 0

    if stim == "Carousel":
        page_height = 2191

    if stim == "Grid" or stim == "List":
        page_height = 2912

    # Przeksztalcenie liniowe
    prop = (page_height / y_res)
    physical_width = x_res / prop
    offset = 0.5*(x_res - physical_width)
    x = ((x-offset)/physical_width) * page_width
    y = (y / y_res) * page_height
    
    return [x, y]

# Metoda do sprawdzania, czy pod przekazanymi koordynatami znajduje sie jakis produkt
def check_coords(x, y, stim):

    x, y = calculate(x, y, stim)

    # Sprawdzanie czy pod wspolrzednymi kryja sie produkty    
    if stim == "Grid":
        
        coords = [
            (600, 950, 800, 1300),
            (1050, 1400, 800, 1300),
            (1500, 1850, 800, 1300),
            (600, 950, 1380, 1880),
            (1050, 1400, 1380, 1880),
            (1500, 1850, 1380, 1880),
            (600, 950, 1960, 2460),
            (1050, 1400, 1960, 2460),
            (1500, 1850, 1960, 2460),
        ]

        i = 1
        for c in coords:
            if (c[0] <= x <= c[1]) and (c[2] <= y <= c[3]): return [x, y, i]
            i +=1
 
        return [x, y, -1]
        
        
    if stim == "List": 
        
        coords = [
            (600, 1800, 800, 1100),
            (600, 1800, 1120, 1420),
            (600, 1800, 1440, 1740),
            (600, 1800, 1760, 2060),
            (600, 1800, 2080, 2380),
        ]

        i = 1
        for c in coords:
            if (c[0] <= x <= c[1]) and (c[2] <= y <= c[3]): return [x, y, i]
            i += 1

        return [x, y, -1]
        
    if stim == "Carousel":
        
        coords = [
            (20, 370, 1086, 1584),
            (390, 740, 1086, 1584),
            (760, 1210, 1086, 1584),
            (1130, 1480, 1086, 1584),
            (1500, 1850, 1086, 1584),
        ]

        i = 1
        for c in coords:
            if (c[0] <= x <= c[1]) and (c[2] <= y <= c[3]): return [x, y, i]
            i += 1

        return [x, y, -1]
        

In [5]:
products = pd.DataFrame(columns=["Participant", "Stimulus", "Product Number"])

# Stworzenie ramki danych z obserwacji uzytkownikow
for person in stats:
    for stim in person[1:]:
        for fix in range(1, stim[1] + 1):
            data_temp = data[(data["Participant"]==person[0]) & (data["Stimulus"]==stim[0]) & (data["Fixation Index"]==fix)]
            x_mean = data_temp["Point of Regard Right X [px]"].median()
            y_mean = data_temp["Point of Regard Right Y [px]"].median()
            x, y, p_id = check_coords(x_mean, y_mean, stim[0])
            
            if not math.isnan(x) and not math.isnan(y): products.loc[len(products.index)] =  [person[0], stim[0], p_id]

products = products[products["Product Number"] != -1].drop_duplicates()
products

,Participant,Stimulus,Product Number
0,SK,List,2
2,SK,List,1
7,SK,List,3
9,SK,List,5
10,SK,List,4
...,...,...,...
524,KU,Carousel,4
525,KU,Carousel,5
527,KU,Grid,5
529,KU,Grid,9


In [6]:
# Przygotowanie danych do wykresow
grid_data_to_histogram  = products[products["Stimulus"] == "Grid"].sort_values(by="Product Number")
list_data_to_histogram  = products[products["Stimulus"] == "List"].sort_values(by="Product Number")
carousel_data_to_histogram  = products[products["Stimulus"] == "Carousel"].sort_values(by="Product Number")

# Wyrysowanie wykresow
fig2 = px.histogram(grid_data_to_histogram, x='Product Number', color='Product Number', nbins=9,
                    title='Liczba użytkowników spoglądających na dany produkt na stronie Grid')

fig3 = px.histogram(list_data_to_histogram, x='Product Number', color='Product Number', nbins=5,
                    title='Liczba użytkowników spoglądających na dany produkt na stronie List')

fig4 = px.histogram(carousel_data_to_histogram, x='Product Number', color='Product Number', nbins=5,
                    title='Liczba użytkowników spoglądających na dany produkt na stronie Carousel')

fig2.update_layout(showlegend=False)    
fig3.update_layout(showlegend=False)    
fig4.update_layout(showlegend=False)    

fig2.show()
fig3.show()
fig4.show()

In [7]:
products_on_grid = 9
products_on_carousel = 5 
products_on_list = 5
experiments = len(products["Participant"].unique())

# Przeliczenie na ile produktow spojrzal kazdy uczestnik
counts = products.groupby(["Participant", "Stimulus"]).agg({"Product Number": "size"}).reset_index()
counts["Count"] = counts["Product Number"]
counts = counts.drop(["Product Number"], axis=1)

counts

,Participant,Stimulus,Count
0,AM,Carousel,2
1,AM,Grid,5
2,AM,List,2
3,AT,Carousel,5
4,AT,Grid,6
5,AT,List,5
6,BM,Grid,2
7,BM,List,2
8,DM,Carousel,4
9,DM,Grid,1


In [8]:
# Wyrysowanie kolejnych wykresow
fig6 = px.bar(counts[counts["Stimulus"] == "Grid"], x="Participant", y="Count", color="Participant", title="Liczba produktów, na które uczestnicy zwrócili uwagę na stronie Grid", labels={"Count": "Ilość produktów", "Participant": "Uczestnik"})

fig7 = px.bar(counts[counts["Stimulus"] == "List"], x="Participant", y="Count", color="Participant", title="Liczba produktów, na które uczestnicy zwrócili uwagę na stronie List", labels={"Count": "Ilość produktów", "Participant": "Uczestnik"})

fig8 = px.bar(counts[counts["Stimulus"] == "Carousel"], x="Participant", y="Count", color="Participant", title="Liczba produktów, na które uczestnicy zwrócili uwagę na stronie Carousel", labels={"Count": "Ilość produktów", "Participant": "Uczestnik"})

fig6.update_layout(showlegend=False)
fig7.update_layout(showlegend=False)
fig8.update_layout(showlegend=False)

fig6.show()
fig7.show()
fig8.show()

In [9]:
# Obliczenie procentowej efektywnosci ukladow
grid_percents = counts[counts["Stimulus"] == "Grid"]["Count"].sum()
grid_percents = grid_percents / experiments # experiments = liczba uczestnikow
grid_percents = str(round((grid_percents /  products_on_grid) * 100, 2)) + "%"

list_percents = counts[counts["Stimulus"] == "List"]["Count"].sum()
list_percents = list_percents / experiments
list_percents = str(round((list_percents / products_on_list) * 100, 2)) + "%"

carousel_percents = counts[counts["Stimulus"] == "Carousel"]["Count"].sum()
carousel_percents = carousel_percents / experiments
carousel_percents = str(round((carousel_percents / products_on_carousel) * 100, 2)) + "%"

In [10]:
percents = pd.DataFrame({"Page": ["Grid", "List", "Carousel"], "Percents": [grid_percents, list_percents, carousel_percents]})
display(percents)

,Page,Percents
0,Grid,37.37%
1,List,65.45%
2,Carousel,67.27%
